# 读取数据

In [146]:
import pandas as pd
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import zero_one_loss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_gaussian_quantiles
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#OneHotEncoder().fit_transform(iris.target.reshape((-1,1)))
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

strain=train
stest=test

# 删除前两列

In [147]:
print(test.shape[0])
train=strain.drop(['Unnamed: 0','PassengerId'],axis=1)
test=stest.drop(['Unnamed: 0','PassengerId'],axis=1)
print(train.columns)
print(test.columns)
print(test.shape[0])
print(test.iloc[1,])

418
Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['Unnamed: 0.1', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
418
Unnamed: 0.1                                   1
Pclass                                         3
Name            Wilkes, Mrs. James (Ellen Needs)
Sex                                       female
Age                                         47.0
SibSp                                          1
Parch                                          0
Ticket                                    363272
Fare                                         7.0
Cabin                                        NaN
Embarked                                       S
Name: 1, dtype: object



# Names 转化

In [148]:
def changename(x):
    y=x['Name'].split(' ')[1]
    y=y[0:len(y)-1]
    if y in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Mr']: return 1#MR is 1
    if y in ['Countess', 'Mme','Mrs']:
        return 2# MRS is 2
    if y in ['Mlle', 'Ms','Miss']:
        return 3#MISS is 3
    if y =='Dr':
        if x['Sex']=='Male':
            return 1
        else:
            return 2
    return 4
train['Name']=train.apply(changename,axis=1)
test['Name']=test.apply(changename,axis=1)
print(test.shape[0])

418


In [149]:
train=train.drop('Sex',axis=1)
test=test.drop('Sex',axis=1)

In [150]:
print(train.iloc[1,:])

Survived           1
Pclass             1
Name               2
Age             38.0
SibSp              1
Parch              0
Ticket      PC 17599
Fare         71.2833
Cabin            C85
Embarked           C
Name: 1, dtype: object


# ticket取最后一位，cabin不要

In [151]:
def lastticket(x):
    y=x['Ticket']
    try:
        return int(y[len(y)-1])
    except:
        return 10
train['Ticket']=train.apply(lastticket, axis=1)
test['Ticket']=test.apply(lastticket, axis=1)

In [152]:
train=train.drop('Cabin',axis=1)
test=test.drop('Cabin',axis=1)
print(test.shape[0])

418


# 去掉名字，给性别和enbark编号

In [153]:
def enbark_code(x):
    if x['Embarked']=='S': return 0
    if x['Embarked']=='Q': return 1
    if x['Embarked']=='C': return 2

train['Embarked']=train.apply(enbark_code,axis=1)

test['Embarked']=test.apply(enbark_code,axis=1)
print(test.shape[0])


418


# 修改nan

In [154]:
def fillem(a):
    for index,i in a.iterrows():
        if np.isnan(i['Embarked']):
            a['Embarked']=0
    return a
print(test.head(5))

   Unnamed: 0.1  Pclass  Name   Age  SibSp  Parch  Ticket     Fare  Embarked
0             0       3     1  34.5      0      0       1   7.8292         1
1             1       3     2  47.0      1      0       2   7.0000         0
2             2       2     1  62.0      0      0       6   9.6875         1
3             3       3     1  27.0      0      0       4   8.6625         0
4             4       3     2  22.0      1      1       8  12.2875         0


In [155]:

train.dropna(axis=0,inplace=True)

test['Embarked']=test.groupby(['Name','Embarked','Pclass'])['Embarked'].transform(lambda x: x.fillna(0))
test['Fare']=test.groupby(['Name','Embarked','Pclass'])['Fare'].transform(lambda x: x.fillna(x.mean()))
test['Age']=test.groupby(['Name','Embarked','Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))

#a.loc[(a['Name']==i['Name'])& (a['Embarked']==i['Embarked']),'Fair'].mean()
print(test.shape[0])
##############################################
#df['Age'] = df.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))



418


In [156]:
test.dropna(axis=0,inplace=True)
print(test.shape[0])

418


# check train

In [157]:
test.drop('Unnamed: 0.1',axis=1,inplace=True)
test.head(5)

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,1,34.5,0,0,1,7.8292,1
1,3,2,47.0,1,0,2,7.0000,0
2,2,1,62.0,0,0,6,9.6875,1
3,3,1,27.0,0,0,4,8.6625,0
4,3,2,22.0,1,1,8,12.2875,0


# 划分train test

In [158]:
tr=train['Survived']
te=train.drop('Survived',axis=1)
x_train,x_test,y_train,y_test=train_test_split(te,tr,test_size=0.25)

print(y_train)

169    0
725    0
491    0
246    0
664    1
      ..
875    1
27     0
479    1
525    0
679    1
Name: Survived, Length: 534, dtype: int64


# LOGIC 随机森林，adaboost

In [159]:
def acc0(a,b):
    num=0
    rig=0
    for i,j in zip(a,b):
        if i==0:
            num+=1
            if j==i:rig+=1
    return rig/num
def acc1(a,b):
    num=0
    rig=0
    for i,j in zip(a,b):
        if i==1:
            num+=1
            if j==i:rig+=1
    return rig/num

In [160]:
from sklearn.linear_model import LogisticRegression
mod1=LogisticRegression(max_iter=1000,multi_class="ovr")
mod2=RandomForestClassifier(n_estimators=80,max_depth=5)
mod3=DecisionTreeClassifier(max_depth=10,min_samples_split=10)
mod4=AdaBoostClassifier(base_estimator=mod3,n_estimators=300,algorithm='SAMME',)

mod1.fit(x_train,y_train)
mod2.fit(x_train,y_train)
mod3.fit(x_train,y_train)
mod4.fit(x_train,y_train)

t1=mod1.predict(x_test)
for i in range(len(t1)):
    if t1[i]>0.5: t1[i]=1
    else: t1[i]=0
t2=mod2.predict(x_test)
t3=mod3.predict(x_test)
t4=mod4.predict(x_test)

print(acc0(t1,y_test),acc1(t1,y_test),sep='  ')
print(acc0(t2,y_test),acc1(t2,y_test),sep='  ')
print(acc0(t3,y_test),acc1(t3,y_test),sep='  ')
print(acc0(t4,y_test),acc1(t4,y_test),sep='  ')



0.751937984496124  0.8367346938775511
0.7734375  0.88
0.775  0.7931034482758621
0.7807017543859649  0.75


In [162]:
from sklearn.model_selection import GridSearchCV
grid1=GridSearchCV(estimator=mod2,
                   param_grid={'n_estimators':range(50,101,10),
                    'max_depth':range(4,12,1),
                    'max_features': [2, 4, 6, 8],
                    'min_samples_leaf':range(8,27,2)},
                   cv=4, scoring='neg_mean_squared_error')

'''grid1=GridSearchCV(estimator=mod2,
                   param_grid={'n_estimators':range(50,101,10),
                               'max_depth':range(4,12,1),
                               'max_features': [2, 4, 6, 8],
                               'min_samples_leaf':range(8,27,2)},
                   cv=4, scoring='neg_mean_squared_error')'''

grid1.fit(te,tr)
print(grid1.best_score_)

-0.175561797752809


In [163]:
print(grid1.best_params_)

{'max_depth': 11, 'max_features': 8, 'min_samples_leaf': 10, 'n_estimators': 70}


In [164]:

ytest=grid1.predict(test)


passger=[]
survived=[]
for i in range(len(ytest)):
    passger.append(892+i)
    survived.append(ytest[i])
per=pd.DataFrame(columns=['PassengerId','Survived'])
per['PassengerId']=passger
per['Survived']=survived
per.to_csv("submit.csv")

In [168]:
grid2=GridSearchCV(estimator=mod4,
                   param_grid={'n_estimators':range(200,501,100),
                    'algorithm':['SAMME','SAMME.R']},
                   cv=4, scoring='neg_mean_squared_error')
grid2.fit(te,tr)

GridSearchCV(cv=4,
             estimator=AdaBoostClassifier(algorithm='SAMME',
                                          base_estimator=DecisionTreeClassifier(max_depth=10,
                                                                                min_samples_split=10),
                                          n_estimators=300),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'n_estimators': range(200, 501, 100)},
             scoring='neg_mean_squared_error')

In [169]:
yytest=grid2.predict(test)


passger=[]
survived=[]
for i in range(len(yytest)):
    passger.append(892+i)
    survived.append(yytest[i])
per=pd.DataFrame(columns=['PassengerId','Survived'])
per['PassengerId']=passger
per['Survived']=survived
per.to_csv("submit1.csv")